In [34]:
from __future__ import absolute_import,division,print_function
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from sklearn.utils import shuffle


In [35]:
def load_data(path):
    folders = os.listdir(path)
    img_data = []
    label_data = []
    c = 0
    for folder in  folders:
        images=os.listdir(path+folder)
        for image in images:
            img = cv2.imread(path+folder+'/'+image)
            
            img = cv2.resize(img , (32,32) ,interpolation=cv2.INTER_AREA)

            label= folder.split("n")[1]
            
            print(img_data.append(img))

            label_data.append(label)

            c = c + 1


    img_data = np.array(img_data)


    label_data=to_myone_hot(label_data)
    label_data=np.argmax(label_data,1)

    img_data, label_data = shuffle(img_data, label_data, random_state=0)
    return img_data, label_data 



In [36]:
def to_myone_hot(label_data):

    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import OneHotEncoder
    from numpy import array
    values=array(label_data)

    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded =  integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    
    return onehot_encoded

In [37]:
def Network(Input): #input : [Batch_size, 32, 32, 1]
    with tf.name_scope("Network"):
       
        conv1_1 = tf.layers.conv2d(Input, filters = 64, kernel_size = 3, strides = 1, padding='SAME', activation = tf.nn.relu, name = 'conv1_1')
        conv1_2 = tf.layers.conv2d(conv1_1, filters = 64, kernel_size = 3, strides = 1, padding='SAME', activation = tf.nn.relu, name = 'conv1_2')
        pool1 = tf.layers.max_pooling2d(conv1_2, pool_size = 2, strides = 2, padding='SAME', name = 'pool1')
        
        conv2_1 = tf.layers.conv2d(pool1, filters = 128, kernel_size = 3, strides = 1, padding='SAME', activation = tf.nn.relu, name = 'conv2_1')
        conv2_2 = tf.layers.conv2d(conv2_1, filters = 128, kernel_size = 3, strides = 1, padding='SAME', activation = tf.nn.relu, name = 'conv2_2')
        pool2 = tf.layers.max_pooling2d(conv2_2, pool_size = 2, strides = 2, padding='SAME', name = 'pool2')

        conv3_1 = tf.layers.conv2d(pool2, filters = 256, kernel_size = 3, strides = 1, padding='SAME', activation = tf.nn.relu, name = 'conv3_1')
        conv3_2 = tf.layers.conv2d(conv3_1, filters = 256, kernel_size = 3, strides = 1, padding='SAME', activation = tf.nn.relu, name = 'conv3_2')
        conv3_3 = tf.layers.conv2d(conv3_2, filters = 256, kernel_size = 3, strides = 1, padding='SAME', activation = tf.nn.relu, name = 'conv3_3')
        pool3 = tf.layers.max_pooling2d(conv3_3, pool_size = 2, strides = 2, padding='SAME', name = 'pool3')
        
        conv4_1 = tf.layers.conv2d(pool3, filters = 512, kernel_size = 3, strides = 1, padding='SAME', activation = tf.nn.relu, name = 'conv4_1')
        conv4_2 = tf.layers.conv2d(conv4_1, filters = 512, kernel_size = 3, strides = 1, padding='SAME', activation = tf.nn.relu, name = 'conv4_2')
        conv4_3 = tf.layers.conv2d(conv4_2, filters = 512, kernel_size = 3, strides = 1, padding='SAME', activation = tf.nn.relu, name = 'conv4_3')
        pool4 = tf.layers.max_pooling2d(conv4_3, pool_size = 2, strides = 2, padding='SAME', name = 'pool4')
        
        conv5_1 = tf.layers.conv2d(pool4, filters = 512, kernel_size = 3, strides = 1, padding='SAME', activation = tf.nn.relu, name = 'conv5_1')
        conv5_2 = tf.layers.conv2d(conv5_1, filters = 512, kernel_size = 3, strides = 1, padding='SAME', activation = tf.nn.relu, name = 'conv5_2')
        conv5_3 = tf.layers.conv2d(conv5_2, filters = 512, kernel_size = 3, strides = 1, padding='SAME', activation = tf.nn.relu, name = 'conv5_3')
    
        
        flat = tf.contrib.layers.flatten(conv5_3)
        fc1 = tf.layers.dense(flat, units = 1024, activation = tf.nn.relu, name = 'fc1')
        fc2 = tf.layers.dense(fc1, units = 256, activation = None, name = 'fc2')
        fc3 = tf.layers.dense(fc2, units = 10, activation = None, name = 'fc3')
        
     
    return fc3
    
    

In [38]:
def initialize_vgg():
    weight_file = 'vgg16_weights.npz'
    weights = np.load(weight_file)
    keys = sorted(weights.keys())
    keys = keys[:-6]
    for i, k in enumerate(keys):
        #print(i, k, np.shape(weights[k]), tf.trainable_variables()[i])
        sess.run(tf.trainable_variables()[i].assign(weights[k]))
    print('Loaded from VGG')

In [39]:
def loss_function(logit, Label):
    
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logit, labels = Label))
    
    return loss

In [40]:
def Accuracy_Evaluate(prediction, Label):
    # Evaluate model
    #correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Label, 1))
    correct_pred = tf.equal(tf.argmax(tf.nn.softmax(prediction), 1), tf.cast(Label, tf.int64))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    return correct_pred, accuracy

In [41]:
def main(train_data, train_label, no_of_epochs = 150000, batchsize = 32):
    
    
    Input = tf.placeholder(dtype = tf.float32, shape = [batchsize,32,32,3])
    Label = tf.placeholder(dtype = tf.int32, shape = [batchsize])
    
    
    logit = Network(Input)
    
    
    prediction = tf.nn.softmax(logit)
    
    loss = loss_function(logit, Label)
    
    correct_pred, accuracy = Accuracy_Evaluate(prediction, Label)
    
    
    optimiz = tf.train.GradientDescentOptimizer(learning_rate = 0.001).minimize(loss)

    init = tf.global_variables_initializer()
    sess.run(init)
    
    initialize_vgg()
 
    
    tf.summary.scalar('Loss_Value',loss)
    tf.summary.scalar('Accuracy',accuracy)
    
    print('Setting up summary op...')
    summary_op = tf.summary.merge_all()
    
    print('Setting Up Saver...')
    summary_writer = tf.summary.FileWriter('./log_dir/', sess.graph)
    
    itr = 0
    for epoch in range(no_of_epochs):
        
        index = np.random.permutation(np.shape(train_data)[0])
        train_data = train_data[index, :, :, :]
        train_label = train_label[index]
        
        for idx in range(train_data.shape[0]//batchsize): 
            
            batchx = train_data[idx*batchsize : (idx + 1)*batchsize , :, :, :]
            batchy = train_label[idx*batchsize : (idx + 1)*batchsize]
            
            feed_dict = {Input : batchx , Label : batchy}
            
            _, train_loss, train_accuracy, summary_str = sess.run([optimiz, loss, accuracy, summary_op] , feed_dict )
            summary_writer.add_summary(summary_str, itr)
            itr = itr + 1
            
            if idx%10 == 0:
                
                print ('epoch : '+str(epoch)+' step : '+str(idx) + ' train_loss : '+str(train_loss) +
                        ' train_accuracy : '+str(train_accuracy) 
            
                      )

In [48]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
global sess

config = tf.ConfigProto()
sess = tf.Session(config = config)
graph = tf.get_default_graph()

In [43]:
train_data, train_label =load_data('/root/Desktop/monkey/training/')

(1358, 2048, 3)
(32, 32, 3)
None
(1166, 1702, 3)
(32, 32, 3)
None
(466, 639, 3)
(32, 32, 3)
None
(900, 607, 3)
(32, 32, 3)
None
(322, 484, 3)
(32, 32, 3)
None
(494, 655, 3)
(32, 32, 3)
None
(742, 989, 3)
(32, 32, 3)
None
(866, 1300, 3)
(32, 32, 3)
None
(1358, 2048, 3)
(32, 32, 3)
None
(396, 594, 3)
(32, 32, 3)
None
(629, 944, 3)
(32, 32, 3)
None
(1280, 2048, 3)
(32, 32, 3)
None
(200, 300, 3)
(32, 32, 3)
None
(480, 852, 3)
(32, 32, 3)
None
(367, 550, 3)
(32, 32, 3)
None
(1365, 2048, 3)
(32, 32, 3)
None
(743, 990, 3)
(32, 32, 3)
None
(999, 1500, 3)
(32, 32, 3)
None
(447, 640, 3)
(32, 32, 3)
None
(608, 448, 3)
(32, 32, 3)
None
(666, 1000, 3)
(32, 32, 3)
None
(370, 470, 3)
(32, 32, 3)
None
(863, 1296, 3)
(32, 32, 3)
None
(1080, 1920, 3)
(32, 32, 3)
None
(720, 540, 3)
(32, 32, 3)
None
(533, 800, 3)
(32, 32, 3)
None
(861, 1293, 3)
(32, 32, 3)
None
(447, 500, 3)
(32, 32, 3)
None
(480, 852, 3)
(32, 32, 3)
None
(527, 360, 3)
(32, 32, 3)
None
(2592, 3888, 3)
(32, 32, 3)
None
(370, 470, 3)
(32, 3

(443, 295, 3)
(32, 32, 3)
None
(468, 700, 3)
(32, 32, 3)
None
(435, 650, 3)
(32, 32, 3)
None
(1118, 1491, 3)
(32, 32, 3)
None
(285, 426, 3)
(32, 32, 3)
None
(600, 800, 3)
(32, 32, 3)
None
(834, 1293, 3)
(32, 32, 3)
None
(895, 1180, 3)
(32, 32, 3)
None
(374, 704, 3)
(32, 32, 3)
None
(295, 562, 3)
(32, 32, 3)
None
(997, 1493, 3)
(32, 32, 3)
None
(468, 700, 3)
(32, 32, 3)
None
(863, 1295, 3)
(32, 32, 3)
None
(600, 800, 3)
(32, 32, 3)
None
(443, 665, 3)
(32, 32, 3)
None
(600, 600, 3)
(32, 32, 3)
None
(400, 600, 3)
(32, 32, 3)
None
(733, 1100, 3)
(32, 32, 3)
None
(500, 800, 3)
(32, 32, 3)
None
(762, 1024, 3)
(32, 32, 3)
None
(295, 446, 3)
(32, 32, 3)
None
(570, 857, 3)
(32, 32, 3)
None
(532, 800, 3)
(32, 32, 3)
None
(599, 900, 3)
(32, 32, 3)
None
(700, 460, 3)
(32, 32, 3)
None
(333, 500, 3)
(32, 32, 3)
None
(422, 633, 3)
(32, 32, 3)
None
(374, 704, 3)
(32, 32, 3)
None
(861, 1297, 3)
(32, 32, 3)
None
(866, 1300, 3)
(32, 32, 3)
None
(500, 800, 3)
(32, 32, 3)
None
(596, 800, 3)
(32, 32, 3)
Non

(1300, 866, 3)
(32, 32, 3)
None
(800, 600, 3)
(32, 32, 3)
None
(520, 399, 3)
(32, 32, 3)
None
(869, 1298, 3)
(32, 32, 3)
None
(680, 1024, 3)
(32, 32, 3)
None
(336, 502, 3)
(32, 32, 3)
None
(436, 650, 3)
(32, 32, 3)
None
(1300, 851, 3)
(32, 32, 3)
None
(777, 653, 3)
(32, 32, 3)
None
(1600, 1239, 3)
(32, 32, 3)
None
(520, 464, 3)
(32, 32, 3)
None
(600, 900, 3)
(32, 32, 3)
None
(417, 650, 3)
(32, 32, 3)
None
(1302, 864, 3)
(32, 32, 3)
None
(470, 646, 3)
(32, 32, 3)
None
(436, 653, 3)
(32, 32, 3)
None
(498, 664, 3)
(32, 32, 3)
None
(488, 366, 3)
(32, 32, 3)
None
(640, 428, 3)
(32, 32, 3)
None
(1024, 683, 3)
(32, 32, 3)
None
(354, 400, 3)
(32, 32, 3)
None
(593, 788, 3)
(32, 32, 3)
None
(767, 1100, 3)
(32, 32, 3)
None
(520, 344, 3)
(32, 32, 3)
None
(900, 600, 3)
(32, 32, 3)
None
(400, 400, 3)
(32, 32, 3)
None
(1411, 1993, 3)
(32, 32, 3)
None
(866, 1299, 3)
(32, 32, 3)
None
(1302, 862, 3)
(32, 32, 3)
None
(450, 299, 3)
(32, 32, 3)
None
(866, 1300, 3)
(32, 32, 3)
None
(900, 600, 3)
(32, 32, 3)

(3648, 5472, 3)
(32, 32, 3)
None
(422, 320, 3)
(32, 32, 3)
None
(389, 592, 3)
(32, 32, 3)
None
(500, 331, 3)
(32, 32, 3)
None
(683, 1024, 3)
(32, 32, 3)
None
(865, 1300, 3)
(32, 32, 3)
None
(599, 398, 3)
(32, 32, 3)
None
(1298, 866, 3)
(32, 32, 3)
None
(601, 736, 3)
(32, 32, 3)
None
(425, 597, 3)
(32, 32, 3)
None
(416, 550, 3)
(32, 32, 3)
None
(650, 433, 3)
(32, 32, 3)
None
(375, 700, 3)
(32, 32, 3)
None
(700, 500, 3)
(32, 32, 3)
None
(750, 501, 3)
(32, 32, 3)
None
(342, 650, 3)
(32, 32, 3)
None
(596, 800, 3)
(32, 32, 3)
None
(867, 1300, 3)
(32, 32, 3)
None
(826, 1200, 3)
(32, 32, 3)
None
(300, 450, 3)
(32, 32, 3)
None
(1005, 1300, 3)
(32, 32, 3)
None
(298, 450, 3)
(32, 32, 3)
None
(439, 439, 3)
(32, 32, 3)
None
(1174, 1200, 3)
(32, 32, 3)
None
(535, 415, 3)
(32, 32, 3)
None
(427, 640, 3)
(32, 32, 3)
None
(369, 660, 3)
(32, 32, 3)
None
(371, 450, 3)
(32, 32, 3)
None
(866, 1300, 3)
(32, 32, 3)
None
(428, 600, 3)
(32, 32, 3)
None
(480, 640, 3)
(32, 32, 3)
None
(585, 750, 3)
(32, 32, 3)
N

(485, 800, 3)
(32, 32, 3)
None
(3240, 4320, 3)
(32, 32, 3)
None
(828, 847, 3)
(32, 32, 3)
None
(533, 800, 3)
(32, 32, 3)
None
(1953, 2984, 3)
(32, 32, 3)
None
(335, 447, 3)
(32, 32, 3)
None
(1445, 1605, 3)
(32, 32, 3)
None
(693, 854, 3)
(32, 32, 3)
None
(1619, 2150, 3)
(32, 32, 3)
None
(4320, 3240, 3)
(32, 32, 3)
None
(706, 846, 3)
(32, 32, 3)
None
(790, 907, 3)
(32, 32, 3)
None
(406, 625, 3)
(32, 32, 3)
None
(480, 600, 3)
(32, 32, 3)
None
(693, 854, 3)
(32, 32, 3)
None
(599, 383, 3)
(32, 32, 3)
None
(2235, 2433, 3)
(32, 32, 3)
None
(2400, 3600, 3)
(32, 32, 3)
None
(866, 1162, 3)
(32, 32, 3)
None
(384, 522, 3)
(32, 32, 3)
None
(334, 451, 3)
(32, 32, 3)
None
(406, 346, 3)
(32, 32, 3)
None
(1193, 858, 3)
(32, 32, 3)
None
(560, 840, 3)
(32, 32, 3)
None
(316, 474, 3)
(32, 32, 3)
None
(1024, 835, 3)
(32, 32, 3)
None
(336, 445, 3)
(32, 32, 3)
None
(600, 400, 3)
(32, 32, 3)
None
(770, 500, 3)
(32, 32, 3)
None
(2400, 3600, 3)
(32, 32, 3)
None
(414, 537, 3)
(32, 32, 3)
None
(683, 1024, 3)
(32, 

'n9153.jpg'

array([[[156, 180, 172],
        [155, 179, 171],
        [154, 178, 170],
        ...,
        [141, 161, 166],
        [147, 166, 169],
        [158, 176, 177]],

       [[154, 177, 172],
        [153, 176, 171],
        [151, 174, 169],
        ...,
        [140, 160, 165],
        [147, 166, 169],
        [155, 173, 174]],

       [[150, 173, 169],
        [150, 173, 169],
        [148, 171, 166],
        ...,
        [138, 158, 163],
        [146, 165, 168],
        [151, 169, 170]],

       ...,

       [[211, 219, 208],
        [207, 216, 206],
        [203, 211, 204],
        ...,
        [217, 233, 222],
        [224, 232, 221],
        [231, 233, 221]],

       [[217, 225, 214],
        [212, 220, 209],
        [204, 213, 203],
        ...,
        [230, 240, 228],
        [233, 235, 223],
        [236, 233, 219]],

       [[228, 237, 224],
        [220, 228, 217],
        [208, 217, 207],
        ...,
        [239, 243, 231],
        [238, 237, 223],
        [242, 233, 219]]

In [49]:
main(train_data, train_label)

Loaded from VGG
Setting up summary op...
Setting Up Saver...
epoch : 0 step : 0 train_loss : 38.82235 train_accuracy : 0.03125
epoch : 0 step : 10 train_loss : 2.2874923 train_accuracy : 0.09375
epoch : 0 step : 20 train_loss : 2.0633779 train_accuracy : 0.25
epoch : 0 step : 30 train_loss : 2.0674796 train_accuracy : 0.3125
epoch : 1 step : 0 train_loss : 1.7375543 train_accuracy : 0.53125
epoch : 1 step : 10 train_loss : 2.2400854 train_accuracy : 0.15625
epoch : 1 step : 20 train_loss : 1.8993624 train_accuracy : 0.3125
epoch : 1 step : 30 train_loss : 1.624648 train_accuracy : 0.34375
epoch : 2 step : 0 train_loss : 1.5888789 train_accuracy : 0.4375
epoch : 2 step : 10 train_loss : 1.8040812 train_accuracy : 0.34375
epoch : 2 step : 20 train_loss : 1.9816607 train_accuracy : 0.25
epoch : 2 step : 30 train_loss : 2.2372465 train_accuracy : 0.21875
epoch : 3 step : 0 train_loss : 1.3529875 train_accuracy : 0.5
epoch : 3 step : 10 train_loss : 1.5211806 train_accuracy : 0.5
epoch : 3 

epoch : 30 step : 10 train_loss : 0.0045481524 train_accuracy : 1.0
epoch : 30 step : 20 train_loss : 0.0011262574 train_accuracy : 1.0
epoch : 30 step : 30 train_loss : 0.005561225 train_accuracy : 1.0
epoch : 31 step : 0 train_loss : 0.0022283965 train_accuracy : 1.0
epoch : 31 step : 10 train_loss : 0.002258006 train_accuracy : 1.0
epoch : 31 step : 20 train_loss : 0.0042035654 train_accuracy : 1.0
epoch : 31 step : 30 train_loss : 0.0021608174 train_accuracy : 1.0
epoch : 32 step : 0 train_loss : 0.0027458845 train_accuracy : 1.0
epoch : 32 step : 10 train_loss : 0.0032542248 train_accuracy : 1.0
epoch : 32 step : 20 train_loss : 0.0025711425 train_accuracy : 1.0
epoch : 32 step : 30 train_loss : 0.0026641479 train_accuracy : 1.0
epoch : 33 step : 0 train_loss : 0.0024669855 train_accuracy : 1.0
epoch : 33 step : 10 train_loss : 0.0037071486 train_accuracy : 1.0
epoch : 33 step : 20 train_loss : 0.0024550362 train_accuracy : 1.0
epoch : 33 step : 30 train_loss : 0.0038059372 train_

KeyboardInterrupt: 